In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle

In [73]:
# reading the dataset
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [74]:
# removing the unnecessary feature
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1)

In [75]:
# encoding the Gender Column using label encoder
LB = LabelEncoder()
df["Gender"] = LB.fit_transform(df["Gender"])

In [76]:
# divide the data into independent and dependent feature
X = df.drop(columns=["Exited"],axis=1)
y = df["Exited"]

In [77]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [86]:
# divide the data into independent and dependent feature
X = df.drop(columns=["Exited"],axis=1)
y = df["Exited"]

In [87]:
# encoding the geography column using column transformer and OHE
Transformer = ColumnTransformer(transformers=[("OHE",OneHotEncoder(drop="first"),["Geography"])],remainder="passthrough")
X = Transformer.fit_transform(X)

In [88]:
with open("label_encoder_gender.pkl","wb") as file:
    pickle.dump(LB,file)

with open("column_transformer_geo.pkl", "wb") as file:
    pickle.dump(Transformer, file)

In [89]:
# doing train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.25)

# scalling the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [90]:
# create a pickel file for sc
with open("standard_scaler.pkl","wb") as file:
    pickle.dump(sc,file)


### ANN implementation

In [91]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [92]:
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ## Hidden Layer 1 connected with input layer
    Dense(32,activation="relu"), ## HL2
    Dense(1,activation="sigmoid") ##Output layer    


]


)

d:\ANN CLASSIFICATION\senv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [93]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
# compile the model for backword propagation

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [95]:
# set up tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [96]:
# set up early stopping
early_stopping_callbacks = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

In [97]:
# train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callbacks]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7975 - loss: 0.4988 - val_accuracy: 0.8344 - val_loss: 0.3965
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8303 - loss: 0.4040 - val_accuracy: 0.8536 - val_loss: 0.3600
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8512 - loss: 0.3635 - val_accuracy: 0.8556 - val_loss: 0.3563
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8652 - loss: 0.3363 - val_accuracy: 0.8564 - val_loss: 0.3485
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8615 - loss: 0.3349 - val_accuracy: 0.8588 - val_loss: 0.3466
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8644 - loss: 0.3344 - val_accuracy: 0.8580 - val_loss: 0.3515
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8571 - loss: 0.3358 - val_accuracy: 0.8580 - val_loss: 0.3478
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8691 - loss: 0.3199 - val_accu

In [98]:
model.save("model.h5")

In [99]:
# Load tensorboard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [100]:
%tensorboard --logdir logs/fit20250403-111349

Reusing TensorBoard on port 6007 (pid 4080), started 0:45:53 ago. (Use '!kill 4080' to kill it.)